# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** if you having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.

* Create a heat map that displays the humidity for every city from the part I of the homework.

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

* For max intensity in the heat map, try setting it to the highest humidity found in the data set.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "Cities weather worldwide.csv"
cities_weather_df = pd.read_csv(csv_path)
cities_weather_df

,Unnamed: 0,Country,Cities,Latitude,Longitude,Minimum temperature,Maximum temperature,Humidity,Cloudiness,Wind Speed
0,0,ZA,East London,-33.02,27.91,76.23,76.23,38,0,5.26
1,1,ZA,Kruisfontein,-34.00,24.73,60.22,60.22,69,1,10.02
2,2,US,Jamestown,42.10,-79.24,57.20,57.20,50,75,16.11
3,3,CA,Tuktoyaktuk,69.45,-133.04,6.80,6.80,92,40,12.75
4,4,EC,Palora,-1.70,-77.93,71.60,71.60,94,90,2.24
...,...,...,...,...,...,...,...,...,...,...
574,574,BR,Salinópolis,-0.61,-47.36,84.61,84.61,72,19,10.38
575,575,IN,Leh,34.17,77.58,45.72,45.72,13,97,2.68
576,576,FR,Vendôme,47.80,1.07,57.20,59.00,67,100,4.70
577,577,CN,Dengzhou,37.81,120.75,61.43,61.43,83,39,8.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store Latitude and Longitude into  locations 
locations = cities_weather_df[["Latitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
cities_weather_df = cities_weather_df.dropna()
cities_humidity_df = cities_weather_df["Humidity"].astype(float)
cities_weather_df

,Unnamed: 0,Country,Cities,Latitude,Longitude,Minimum temperature,Maximum temperature,Humidity,Cloudiness,Wind Speed
0,0,ZA,East London,-33.02,27.91,76.23,76.23,38,0,5.26
1,1,ZA,Kruisfontein,-34.00,24.73,60.22,60.22,69,1,10.02
2,2,US,Jamestown,42.10,-79.24,57.20,57.20,50,75,16.11
3,3,CA,Tuktoyaktuk,69.45,-133.04,6.80,6.80,92,40,12.75
4,4,EC,Palora,-1.70,-77.93,71.60,71.60,94,90,2.24
...,...,...,...,...,...,...,...,...,...,...
574,574,BR,Salinópolis,-0.61,-47.36,84.61,84.61,72,19,10.38
575,575,IN,Leh,34.17,77.58,45.72,45.72,13,97,2.68
576,576,FR,Vendôme,47.80,1.07,57.20,59.00,67,100,4.70
577,577,CN,Dengzhou,37.81,120.75,61.43,61.43,83,39,8.93


In [13]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
fig = gmaps.figure(center = (20,7), zoom_level = 2)
heat_map = gmaps.heatmap_layer(locations, weights=cities_humidity_df, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
vacation_city_df = cities_weather_df.copy()
vacation_city_df = vacation_city_df[vacation_city_df["Minimum temperature"] >= 68]
vacation_city_df = vacation_city_df[vacation_city_df["Maximum temperature"] <= 104]  
vacation_city_df = vacation_city_df[vacation_city_df['Cloudiness'] == 0]
vacation_city_df = vacation_city_df[vacation_city_df['Humidity'] < 30]
vacation_city_df = vacation_city_df[vacation_city_df['Wind Speed'] < 8]
vacation_city_df = vacation_city_df.dropna()
vacation_city_df

,Unnamed: 0,Country,Cities,Latitude,Longitude,Minimum temperature,Maximum temperature,Humidity,Cloudiness,Wind Speed
6,6,IN,Bārān,25.10,76.52,92.66,92.66,11,0,4.94
117,117,MX,San Patricio,19.22,-104.70,89.76,89.76,26,0,5.91
175,175,MX,Ocampo,21.65,-101.50,73.04,73.04,17,0,2.77
280,280,EG,Aswān,24.09,32.91,90.34,90.34,13,0,6.29
374,374,AR,San Vicente,-26.62,-54.13,72.82,72.82,27,0,4.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = vacation_city_df
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,Country,Cities,Latitude,Longitude,Minimum temperature,Maximum temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
6,6,IN,Bārān,25.10,76.52,92.66,92.66,11,0,4.94,
117,117,MX,San Patricio,19.22,-104.70,89.76,89.76,26,0,5.91,
175,175,MX,Ocampo,21.65,-101.50,73.04,73.04,17,0,2.77,
280,280,EG,Aswān,24.09,32.91,90.34,90.34,13,0,6.29,
374,374,AR,San Vicente,-26.62,-54.13,72.82,72.82,27,0,4.43,


In [16]:
g_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {
            "types": "hotel",
            "radius": 5000,
            "key": g_key}

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    parameters["location"] = f"{latitude},{longitude}"
    
    print(f"Retrieving Results for Index {index}: {row['Cities']}.")
    response = requests.get(g_url, params=parameters).json()
  
    results = response["results"]
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing result... skipping.")

Retrieving Results for Index 6: Bārān.
Closest hotel is Baran.
Retrieving Results for Index 117: San Patricio.
Closest hotel is San Patricio.
Retrieving Results for Index 175: Ocampo.
Closest hotel is Ocampo.
Retrieving Results for Index 280: Aswān.
Closest hotel is Aswan.
Retrieving Results for Index 374: San Vicente.
Closest hotel is San Pedro.


In [17]:
hotel_df

,Unnamed: 0,Country,Cities,Latitude,Longitude,Minimum temperature,Maximum temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
6,6,IN,Bārān,25.10,76.52,92.66,92.66,11,0,4.94,Baran
117,117,MX,San Patricio,19.22,-104.70,89.76,89.76,26,0,5.91,San Patricio
175,175,MX,Ocampo,21.65,-101.50,73.04,73.04,17,0,2.77,Ocampo
280,280,EG,Aswān,24.09,32.91,90.34,90.34,13,0,6.29,Aswan
374,374,AR,San Vicente,-26.62,-54.13,72.82,72.82,27,0,4.43,San Pedro


In [18]:
# NOTE: Do not change any of the code in this cell
hotel_locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(hotel_locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(hotel_locations, scale = 2,info_box_content = hotel_info)

# Display Map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))